# Hecho servicios acumulados

In [18]:
from datetime import date
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np
import holidays
import yaml

### Conexión con la base de datos

In [27]:
# Cargar configuraciones
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

# Crear URLs de conexión
url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")

# Crear las conexiones
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

# Función para probar la conexión con las bases de datos
def comprobar_conexionBD(db_engine, db_name):
  try:
    with db_engine.connect() as connection:
      if connection.execute(text("SELECT 1")).fetchone() is not None:
        print(f"Conexión exitosa a base de datos {db_name}")
      else:
        print(f"No se pudo conectar a {db_name}")
  except SQLAlchemyError as e:
    print(f"Error al conectar a base de datos {db_name}: {e}")

comprobar_conexionBD(oltp_conn, config_oltp['dbname'])
comprobar_conexionBD(olap_conn, config_olap['dbname'])


Conexión exitosa a base de datos mensajeriaOLTP
Conexión exitosa a base de datos mensajeriaOLAP


### Módulo de extracción

In [125]:
table_servicio = pd.read_sql_table('mensajeria_servicio', oltp_conn)
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn)
table_estados = pd.read_sql_table('mensajeria_estado', oltp_conn)
dimension_fecha = pd.read_sql('dim_fecha', olap_conn)
dimension_fecha['hora'] = dimension_fecha['hora'].astype(str).str[:5]
hecho_servicios = table_servicio[['id', 'cliente_id', 'mensajero_id', 'ciudad_destino_id', 'ciudad_origen_id', 'mensajero2_id', 'mensajero3_id']]

estados = [
  (1, 'id_fecha_iniciado'),
  (2, 'id_fecha_mensajero'),
  (3, 'id_fecha_recogida'),
  (4, 'id_fecha_entrega'),
  (5, 'id_fecha_terminado')]


for estado_id, nombre in estados: # genera todas las llaves con la dimensión fecha 
  filtered_estados = (table_estadosservicio[['estado_id', 'servicio_id', 'fecha', 'hora']]
    .query(f"estado_id == {estado_id}")
    .drop(columns=['estado_id'])
    .drop_duplicates(subset=['servicio_id']))
  
  filtered_estados['hora'] = filtered_estados['hora'].astype(str).str[:5]

  filtered_estados = (pd.merge(filtered_estados, dimension_fecha[['fecha', 'hora', 'id']],
    how='left', 
    on=['fecha', 'hora'])
    .rename(columns={'id': nombre})
    .drop(columns=['fecha', 'hora']))

  """
  """
  #print(filtered_estados.sort_values(by=['servicio_id']).head(2))

  hecho_servicios = (hecho_servicios
    .merge(filtered_estados, how='left', left_on='id', right_on='servicio_id')
    .drop(columns=['servicio_id'])
    .astype('Int32')
    .replace({pd.NA: None}))

hecho_servicios.sort_values(by=['id']).head(20)
#print(hecho_servicios.shape[0])


,id,cliente_id,mensajero_id,ciudad_destino_id,ciudad_origen_id,mensajero2_id,mensajero3_id,id_fecha_iniciado,id_fecha_mensajero,id_fecha_recogida,id_fecha_entrega,id_fecha_terminado
28324,7,5,1,1,1,7,None,301,667,None,686,687
16,8,5,7,1,1,None,None,302,792,None,5902,34482
2112,9,5,7,1,1,None,None,302,843,None,None,None
28386,10,5,7,1,1,None,None,305,843,None,7386,18045
28385,11,5,7,1,1,None,None,306,766,None,2153,None
28323,12,5,9,1,1,7,None,307,659,None,680,681
28322,13,5,7,1,1,None,None,308,658,None,669,876
28320,14,5,7,1,1,None,None,309,657,None,9523,9952
18,15,5,7,1,1,None,None,310,656,None,None,None
28399,16,5,7,1,1,None,None,311,655,None,878,878


### Módulo de transformación

### Módulo de carga a la bodega de datos

In [110]:
hecho_servicios.set_index('id', inplace=True)
try:
  filtered_estados.to_sql('hecho_servicios', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")